# Advanced Scoring of Graded Readers
## DS 7337 - Natural Language Processing - Homework 2
### George C. Sturrock
### January 26, 2019
#### Homework 2 Objectives
1.	In Python, create a method for scoring the vocabulary size of a text, and normalize the score from 0 to 1. It does not matter what method you use for normalization as long as you explain it in a short paragraph. 
2.	After consulting section 3.2 in chapter 1 of Bird-Klein, create a method for scoring the long-word vocabulary size of a text, and likewise normalize (and explain) the scoring as in step 1 above.
3.	Now create a “text difficulty score” by combining the lexical diversity score from homework 1, and your normalized score of vocabulary size and long-word vocabulary size, in equal weighting. Explain what you see when this score is applied to same graded texts used in homework 1.

#### Import Libraries

In [1]:
import platform; print("Platform:", platform.platform())
import os
import sys; print("Python:", sys.version)
import requests; print("Requests:", requests.__version__)

import nltk; print("NLTK Version:", nltk.__version__)
from nltk.corpus import stopwords
theStopWords = set(stopwords.words('english'))

import numpy as np; print("Numpy Version:", np.__version__)
import pandas as pd; print("Pandas Version:", pd.__version__)
import matplotlib.pyplot as plt; #print("Matplotlib Pyplot Version:", plt.__version__)
import string; #print("String Version:", string.__version__)

import seaborn as sns; print("Seaborn Version:", sns.__version__)
sns.set(style='darkgrid')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup; print("BeatifulSoup Version: 4.6.3")

Platform: Linux-4.15.0-1026-gcp-x86_64-with-debian-stretch-sid
Python: 3.5.6 |Anaconda custom (64-bit)| (default, Aug 26 2018, 21:41:56) 
[GCC 7.3.0]
Requests: 2.19.1
NLTK Version: 3.3
Numpy Version: 1.15.2
Pandas Version: 0.23.4
Seaborn Version: 0.9.0
BeatifulSoup Version: 4.6.3


##### To Do:
1.  Remove licensing
2.  find correct a hrefs and automatically build list of links and download books
3.  Navigate to .txt files for each book

#### Create NLP Pipeline Function

def nlp_pipeline(url):
    #Download source web page, strip HTML if necessary, trim to desired content
    html = urlopen(url[0]).read()
    html = html.decode("utf-8")
    raw = BeautifulSoup(html, "html.parser").get_text(strip=True).lower()
    position, raw_text_without_license = remove_gutenberg_license(raw)
    
    #tokenize the text including the header information
    tokens = nltk.work_tokenize

In [2]:
#Open one text
from urllib import request

url = "http://www.gutenberg.org/cache/epub/15659/pg15659.txt" #The beacon second reader
html = request.urlopen(url).read().decode('utf8')

In [54]:
#print contents of html
print(type(html))
print(html)

<class 'str'>
﻿The Project Gutenberg EBook of The Beacon Second Reader, by James H. Fassett

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: The Beacon Second Reader

Author: James H. Fassett

Illustrator: Edna T. Hart

Release Date: April 19, 2005 [EBook #15659]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK THE BEACON SECOND READER ***




Produced by Mark C. Orton, Melissa Er-Raqabi and the Online
Distributed Proofreading Team at http://www.pgdp.net.






THE
BEACON SECOND READER

BY
JAMES H. FASSETT

GINN AND COMPANY
BOSTON - NEW YORK - CHICAGO - LONDON
ATLANTA - DALLAS - COLUMBUS - SAN FRANCISCO


COPYRIGHT, 1914, BY JAMES H. FASSETT
ALL RIGHTS RESERVED

PRINTED IN THE UNITED STATES OF AMERICA
431.1

The Athenæum Press

GINN AND COMPANY - PROPRIETORS -
BOSTO

In [20]:
print(html.splitlines())

['\ufeffThe Project Gutenberg EBook of The Beacon Second Reader, by James H. Fassett',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.net',
 '',
 '',
 'Title: The Beacon Second Reader',
 '',
 'Author: James H. Fassett',
 '',
 'Illustrator: Edna T. Hart',
 '',
 'Release Date: April 19, 2005 [EBook #15659]',
 '',
 'Language: English',
 '',
 '',
 '*** START OF THIS PROJECT GUTENBERG EBOOK THE BEACON SECOND READER ***',
 '',
 '',
 '',
 '',
 'Produced by Mark C. Orton, Melissa Er-Raqabi and the Online',
 'Distributed Proofreading Team at http://www.pgdp.net.',
 '',
 '',
 '',
 '',
 '',
 '',
 'THE',
 'BEACON SECOND READER',
 '',
 'BY',
 'JAMES H. FASSETT',
 '',
 'GINN AND COMPANY',
 'BOSTON - NEW YORK - CHICAGO - LONDON',
 'ATLANTA - DALLAS - COLUMBUS - SAN FRANCISCO',
 '',
 '',


In [55]:
#split line of html and remove licensing
## credit to: https://stackoverflow.com/questions/20461847/str-startswith-with-a-list-of-strings-to-test-for
html2 = html.splitlines()
print("Text Title:", html2[0])
#print(html2.index("*** START OF THIS PROJECT GUTENBERG EBOOK THE BEACON SECOND READER ***")) #no match returns ValueError
#start = [i for i, s in enumerate(html2) if s.startswith("*** START OF THIS PROJECT GUTENBERG EBOOK")]
start = [i for i, s in enumerate(html2) if s.startswith("CONTENTS")]
end = [i for i, s in enumerate(html2) if s.startswith("*** END OF THIS PROJECT GUTENBERG EBOOK")]
print(start[0], end[0])
text = html2[(start[0]+1):(end[0]-2)]
print("Type:", type(text), "Length:", len(text))
print(text)

Text Title: ﻿The Project Gutenberg EBook of The Beacon Second Reader, by James H. Fassett
Type: <class 'list'>
93 4135
Type: <class 'list'> Length: 4039
['', '', '                                                              PAGE', 'THE SHIP                            _Old English Rhyme_         13', 'THE WOLF AND THE SEVEN YOUNG KIDS   _William and Jacob Grimm_   14', "THEY DIDN'T THINK                   _Phoebe Cary_               22", 'TOM THUMB                           _English Fairy Tale_        24', 'SUPPOSE                             _Alice Cary_                34', 'CINDERELLA                          _English Fairy Tale_        36', 'RAINDROPS                           _Ann Hawkshawe_             43', 'THE FOUR FRIENDS                    _William and Jacob Grimm_   44', 'LITTLE BIRDIE                       _Alfred Tennyson_           54', 'MOTHER FROST                        _William and Jacob Grimm_   55', 'IF EVER I SEE                       _Lydia Maria Child_         65'

In [57]:
len(str(text))

128731

In [61]:
#Strip New Line Characters and convert to lower case
strip = list()

for t in text:
    strip.append(t.rstrip().lower())

print(type(strip))
print(str(strip))

<class 'list'>
['', '', '                                                              page', 'the ship                            _old english rhyme_         13', 'the wolf and the seven young kids   _william and jacob grimm_   14', "they didn't think                   _phoebe cary_               22", 'tom thumb                           _english fairy tale_        24', 'suppose                             _alice cary_                34', 'cinderella                          _english fairy tale_        36', 'raindrops                           _ann hawkshawe_             43', 'the four friends                    _william and jacob grimm_   44', 'little birdie                       _alfred tennyson_           54', 'mother frost                        _william and jacob grimm_   55', 'if ever i see                       _lydia maria child_         65', "why the bear's tail is short        _german folk tale_          66", 'rumpelstiltskin                     _william and jacob grimm_   7

In [86]:
table = str.maketrans('', '', string.punctuation)

#tokenize text
raw = nltk.word_tokenize(str(strip))
raw2 = [r.translate(table) for r in raw]
rawx = [rx for rx in raw2 if rx not in theStopWords and rx.isalpha()]
longword = 0
for r in rawx:
    if len(r) > 
raw3 = nltk.Text(rawx)
raw4 = raw3.tokens
raw4

['page',
 'ship',
 'old',
 'english',
 'rhyme',
 'wolf',
 'seven',
 'young',
 'kids',
 'william',
 'jacob',
 'grimm',
 'nt',
 'think',
 'phoebe',
 'cary',
 'tom',
 'thumb',
 'english',
 'fairy',
 'tale',
 'suppose',
 'alice',
 'cary',
 'cinderella',
 'english',
 'fairy',
 'tale',
 'raindrops',
 'ann',
 'hawkshawe',
 'four',
 'friends',
 'william',
 'jacob',
 'grimm',
 'little',
 'birdie',
 'alfred',
 'tennyson',
 'mother',
 'frost',
 'william',
 'jacob',
 'grimm',
 'ever',
 'see',
 'lydia',
 'maria',
 'child',
 'bear',
 'tail',
 'short',
 'german',
 'folk',
 'tale',
 'rumpelstiltskin',
 'william',
 'jacob',
 'grimm',
 'bed',
 'summer',
 'robert',
 'louis',
 'stevenson',
 'golden',
 'touch',
 'greek',
 'myth',
 'meadow',
 'olive',
 'wadsworth',
 'bell',
 'atri',
 'german',
 'folk',
 'tale',
 'baby',
 'hugh',
 'miller',
 'bruce',
 'spider',
 'scottish',
 'tradition',
 'wise',
 'little',
 'pig',
 'anonymous',
 'indian',
 'story',
 'good',
 'play',
 'robert',
 'louis',
 'stevenson',
 'dick

In [80]:
#remove stopwords, blanks and other 
words = []

for r in raw4:
    if r not in theStopWords:
        if r not in '':
            if r.isalpha():
                words.append(r)

                
print(type(words))
print(len(words))
print(words)
word2 = str(words)
tokens = nltk.word_tokenize(word2)
print("==========================================================================")
print("==========================================================================")
print(type(tokens))
print(len(tokens))
print(tokens)


<class 'list'>
11430
['page', 'ship', 'old', 'english', 'rhyme', 'wolf', 'seven', 'young', 'kids', 'william', 'jacob', 'grimm', 'nt', 'think', 'phoebe', 'cary', 'tom', 'thumb', 'english', 'fairy', 'tale', 'suppose', 'alice', 'cary', 'cinderella', 'english', 'fairy', 'tale', 'raindrops', 'ann', 'hawkshawe', 'four', 'friends', 'william', 'jacob', 'grimm', 'little', 'birdie', 'alfred', 'tennyson', 'mother', 'frost', 'william', 'jacob', 'grimm', 'ever', 'see', 'lydia', 'maria', 'child', 'bear', 'tail', 'short', 'german', 'folk', 'tale', 'rumpelstiltskin', 'william', 'jacob', 'grimm', 'bed', 'summer', 'robert', 'louis', 'stevenson', 'golden', 'touch', 'greek', 'myth', 'meadow', 'olive', 'wadsworth', 'bell', 'atri', 'german', 'folk', 'tale', 'baby', 'hugh', 'miller', 'bruce', 'spider', 'scottish', 'tradition', 'wise', 'little', 'pig', 'anonymous', 'indian', 'story', 'good', 'play', 'robert', 'louis', 'stevenson', 'dick', 'whittington', 'english', 'folk', 'tale', 'new', 'moon', 'eliza', 'lee'

### Saved Code

In [9]:
### Turn web page to text ###
try:
    del raw, raw2, raw3, raw4, raw5
except Exception as e:
    print(e)

table = str.maketrans('', '', string.punctuation)

#get text out of HTML
raw = BeautifulSoup(html).get_text()
raw2 = nltk.word_tokenize(raw)
raw3 = [r.translate(table) for r in raw2]
raw4 = nltk.Text(raw3)
raw5 = raw4.tokens

text = []

#remove stopwords, blanks and other 
for r in raw5:
    if r not in theStopWords:
        if r not in '':
            if r.isalpha():
                text.append(r)
            
print(text)
print(len(text))

name 'raw6' is not defined
['icon', 'background', 'transparent', 'url', 'picsspritepng', 'norepeat', 'CDATA', 'var', 'jsonsearch', 'ebookssuggest', 'var', 'mobileurl', 'var', 'canonicalurl', 'http', 'var', 'lang', 'enUS', 'var', 'fblang', 'enUS', 'FB', 'accepts', 'xxXX', 'var', 'msgloadmore', 'Load', 'More', 'var', 'pagemode', 'screen', 'var', 'dialogtitle', 'var', 'dialogmessage', 'The', 'Beacon', 'Second', 'Reader', 'James', 'H', 'Fassett', 'Free', 'Ebook', 'qrcode', 'background', 'transparent', 'url', 'norepeat', 'Project', 'Gutenberg', 'free', 'ebooks', 'The', 'Beacon', 'Second', 'Reader', 'James', 'H', 'Fassett', 'No', 'cover', 'available', 'Download', 'Bibrec', 'Bibliographic', 'Record', 'Author', 'Fassett', 'James', 'H', 'James', 'Hiram', 'Title', 'The', 'Beacon', 'Second', 'Reader', 'Language', 'English', 'LoC', 'Class', 'PE', 'Language', 'Literatures', 'English', 'Subject', 'Fairy', 'tales', 'Subject', 'Readers', 'Subject', 'Reading', 'Elementary', 'Category', 'Text', 'EBookNo